In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from nlptools.preprocessing import *
from nlptools.basics import *
from twinews.utils import *
from twinews.models.ranking import *
from twinews.evaluation.utils import *
from machinelearning.iterator import *

In [3]:
import re
import codecs
import math
from gensim import corpora
from gensim.summarization import bm25
from nltk.stem import WordNetLemmatizer

In [4]:
modelName = 'BM25_yf'

In [5]:
isNotebook = '__file__' not in locals()

In [6]:
TEST = isNotebook

In [7]:
logger = Logger(tmpDir('logs') + "/bm25.log") if isNotebook else Logger("bm25-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

# BM25 Config

In [57]:
config = \
{
    'splitVersion': 2,
    
    # BM25 free smoonthing para
    'k1': 1.8 if TEST else 0,
    'b': 0.75 if TEST else 0,
    'epsilon': 0.25, # get idf when < 0
    
    'maxUsers': 30, # Sub-sampling
    'maxDocuments': 10000,
    'useExtraNews': False, # None = unlimited, 0 = no extra news
    'minDF': 1 / 2000, # Remove words that have a document frequency ratio lower than 1 / 500
    'maxDF': 300, # Remove top 300 voc elements
    
    'lowercase': False,
    'doLemmatization': True,
    
    'implementation': 'BM25', # bm25f, topic+bm25

}

# Data

In [9]:
evalData = getEvalData(config['splitVersion'], maxExtraNews=config['maxDocuments'],
                       maxUsers=config['maxUsers'], logger=logger)
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

--> tic: 31.59s | message: Eval data loaded
twinews news (version 1.0) initialised.
--> tic: 13.08s | message: Extra news downloaded
--> toc total duration: 44.67s | message: Got Twinews evaluation data
{ candidates, extraNews, meta, testNews, testUsers, trainNews, trainUsers }
{ 'created': 2020.03.24-14.28.06, 'endDate': 2018-01-15, 'id': 2, 'ranksLength': 1000, 'splitDate': 2017-12-25, 'startDate': 2017-10-01, 'testMaxNewsPerUser': 97, 'testMeanNewsPerUser': 7.22, 'testMinNewsPerUser': 2, 'testNewsCount': 71781, 'totalNewsAvailable': 570210, 'trainMaxNewsPerUser': 379, 'trainMeanNewsPerUser': 26.48, 'trainMinNewsPerUser': 8, 'trainNewsCount': 237150, 'usersCount': 15905 }


In [10]:
bp(trainUsers,2)

{
  1198644368: 
  {
    http://bit.ly/2AyDgle: 1512154872,
    http://bit.ly/2BFx3UT: 1513695568,
    http://bit.ly/2DK0R02: 1512588091,
    http://bit.ly/2gU0G8X: 1510764047,
    http://bit.ly/2kBvCzV: 1513002012,
    ...,
    https://www.nytimes.com/interactive/2017/12/14/opinion/sunday/trump-lies-obama-who-is-worse.html?action=click&pgtype=Homepage&clickSource=story-heading&module=opinion-c-col-left-region&region=opinion-c-col-left-region&WT.nav=opinion-c-col-left-region: 1513284806,
    https://www.texastribune.org/2017/11/13/texas-capitol-victims-sexual-harassment-must-fend-themselves/: 1510764264,
    https://www.theatlantic.com/politics/archive/2017/10/one-more-straw-upon-the-camels-back/543957/?utm_source=twb: 1508950894,
    https://www.vox.com/policy-and-politics/2017/12/12/16762032/trump-moore-republicans-taxes-lies: 1513093683,
    https://www.washingtonpost.com/news/democracy-post/wp/2017/10/17/how-the-russians-pretended-to-be-texans-and-texans-believed-them/?tid=sm_fb&ut

In [11]:
# extraNewsList = shuffle(list(extraNews), seed=0)
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)
print("nb of trainNews : %d" % len(trainNewsList))
print("nb of testNews : %d" % len(testNewsList))
#print("nb of extraNews : %d" % len(extraNewsList))

nb of trainNews : 710
nb of testNews : 24544


In [12]:
usersList = list(evalData["candidates"].keys())
bp(usersList, 2)
len(usersList)

[ 790664132, 2617447752, ..., 40872545, 729623 ]


30

In [13]:
# build the query and corpus dict(here the user trainNews as history urls for query)
NewsList = trainNewsList + testNewsList
# Print all:
log(str(len(NewsList)) + " urls for NewsList:\n" + b(NewsList), logger=logger)

25254 urls for NewsList:
[
  http://bit.ly/2zSOKMr,
  http://wapo.st/2ihMz1k?tid=ss_tw&utm_term=.8329d8da008c,
  ...,
  https://www.adl.org/blog/white-supremacists-praise-trumps-inflammatory-immigration-remarks,
  http://ftw.usatoday.com/2017/12/georgia-bulldogs-football-rose-bowl-oklahoma-lawyer-argument-change-
]


In [14]:
# get sentences
sentences = getNewsSentences(NewsList, logger=logger)
bp(sentences, logger=logger)

twinews news (version 1.0) initialised.
  0% [                    ]
  9% [=                   ] (12m 34.962s left)
 19% [===                 ] (10m 56.77s left)
 29% [=====               ] (9m 33.056s left)
 39% [=======             ] (8m 9.189s left)
 49% [=========           ] (6m 46.888s left)
 59% [===========         ] (5m 23.901s left)
 69% [=============       ] (4m 1.876s left)
 79% [===============     ] (2m 41.82s left)
 89% [=================   ] (1m 20.737s left)
 99% [=================== ] (0.127s left)
100% [====================] (total duration: 13m 24.73s, mean duration: 0.031s)
[
  [ [ BRENTWOOD, ,, ..., man, . ], [ Nashville, police, ..., couple, . ], ..., [ White, nationalist, ..., Shelbyville, . ], [ A, second, ..., canceled, . ] ],
  [ [ Daphne, Caruana, ..., car, . ], [ (, Malta, ..., ablaze, . ], ..., [ In, blog, ..., companies, . ], [ She, alleged, ..., state, . ] ],
  ...,
  [ [ White, supremacists, ..., countries, . ], [ Mr., Trump, ..., Times, . ], ..., [ Pan

In [15]:
# flatten the senctence
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

[ [ BRENTWOOD, ,, ..., canceled, . ], [ Daphne, Caruana, ..., state, . ], ..., [ White, supremacists, ..., tweet, : ], [ This, might, ..., (, p.m. ] ]


In [16]:
# Lower case:
if config['lowercase']:
    for i in pb(list(range(len(docs))), logger=logger, message="Lower casing"):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)
 

[ [ BRENTWOOD, ,, ..., canceled, . ], [ Daphne, Caruana, ..., state, . ], ..., [ White, supremacists, ..., tweet, : ], [ This, might, ..., (, p.m. ] ]


In [17]:
# Lemmatization:
if config['doLemmatization']:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

Lemmatization   0% [                    ]
Lemmatization   9% [=                   ] (1m 13.272s left)
Lemmatization  19% [===                 ] (54.61s left)
Lemmatization  29% [=====               ] (44.88s left)
Lemmatization  39% [=======             ] (37.059s left)
Lemmatization  49% [=========           ] (30.119s left)
Lemmatization  59% [===========         ] (23.756s left)
Lemmatization  69% [=============       ] (17.619s left)
Lemmatization  79% [===============     ] (11.661s left)
Lemmatization  89% [=================   ] (5.785s left)
Lemmatization  99% [=================== ] (0.009s left)
Lemmatization 100% [====================] (total duration: 57.72s, mean duration: 0.002s)
[ [ BRENTWOOD, ,, ..., canceled, . ], [ Daphne, Caruana, ..., state, . ], ..., [ White, supremacist, ..., tweet, : ], [ This, might, ..., (, p.m. ] ]


In [18]:
# Filtering the corpus:
docs = filterCorpus(docs, minDF=config['minDF'], maxDF=config['maxDF'],
                    removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
for doc in docs: assert len(doc) > 0
bp(docs, logger)

Voc removed because of minDF (158310 elements):
{ 0, 00s, 019568429X, 05CM04, 0600h, 0640L, 0F, 0cean, 0comments, 0day, ..., 😍, 😐, 😒, 😠, 😡, 😢, 😫, 😱, 🙅, 🙌 }
Voc removed because of maxDF (300 elements):
{ ", ', (, ), ,, -, ., :, ;, ?, ..., will, with, without, work, working, world, would, year, you, your }
83.05% of voc will be removed.
[ [ Tenn., AP, ..., Brentwood, canceled ], [ Daphne, journalist, ..., Soviet, satellite ], ..., [ White, supremacist, ..., photo, tweet ], [ sport, related, ..., Monday, p.m. ] ]


# BM25 ranking

In [58]:
usersList = list(evalData['candidates'].keys())

In [59]:
chunksSize = int(len(usersList) / 500)
if chunksSize == 0:
    chunksSize = 1

In [60]:
usersList = chunks(usersList, chunksSize)
bp(usersList, 3, logger)

[ [ 790664132 ], [ 2617447752 ], [ 62594694 ], [ 839264223741956096 ], [ 2786961721 ], ..., [ 15984413 ], [ 621359253 ], [ 1447457215 ], [ 40872545 ], [ 729623 ] ]


In [61]:
urlDocs = dict()
for i in range(len(NewsList)):
    urlDocs[NewsList[i]] = docs[i]

bp(urlDocs, 2, logger)

{
  http://247wallst.com/healthcare-business/2017/12/30/30-big-biotech-events-coming-in-2018/3/: [ Co., NASDAQ, ..., target, __float_2__ ],
  http://247wallst.com/special-report/2017/12/08/americas-25-dying-industries-2: [ An, estimated, ..., dying, industry ],
  http://247wallst.com/special-report/2018/01/11/25-best-fitness-tech-gadgets/: [ Americans, looking, ..., revenue, sale ],
  http://a.msn.com/00/en-us/AAtmQOZ?ocid=st: [ America, live, ..., usage, rate ],
  http://a.msn.com/00/en-us/AAuyJLc?ocid=st: [ Wal, Mart, ..., enough, meet ],
  ...,
  https://xtinaluvspink.wordpress.com/2016/01/17/vegan-chickpea-barley-soup-for-two/: [ received, free, ..., vegetable, stock ],
  https://yaledailynews.com/blog/2018/01/06/yale-psychiatrist-met-with-congressmen-about-trumps-mental-health/: [ School, Medicine, ..., elected, office ],
  https://zdubbzattmom.wordpress.com/2018/01/03/ex-libris-the-eyes-of-madness-presents-2017s-bakers-dozen-zakks-favorite-reads-of-the-year/: [ Ex, Eyes, ..., sto

In [62]:
def genFunct(containers, *args, **kwargs):
    if not isinstance(containers[0], list):
        containers = [containers]
    for container in containers:
        for userId in container:
            request = []
            for url in evalData['trainUsers'][userId]:
                request += urlDocs[url] # the request is the sum historical url in trainUser
            currentRankings = []
            for candidates in evalData['candidates'][userId]:
                candidates = list(candidates) # list of each 1000 candidates for one user
                urlCorpus = dict()
                for i in range(len(candidates)):
                    urlCorpus[candidates[i]] = urlDocs[candidates[i]] # candidate dict, key:url, value:news doc
                urlCorpus = list(urlCorpus.items()) # list of url->news doc
                corpus = [e[1] for e in urlCorpus] # get rid of the url, take the whole coprus
                try:
                    # This is the new version not yet pushed on pipy:
                    model = bm25.BM25(corpus, k1=config['k1'], b=config['b'], epsilon=config['epsilon'])
                except:
                    # This is the actual version with parameters as global variables:
                    bm25.PARAM_K1 = config['k1']
                    bm25.PARAM_B = config['b']
                    bm25.EPSILON = config['epsilon']
                    model = bm25.BM25(corpus)
                scores = model.get_scores(request)
                scoresWithUrl = []
                for i in range(len(scores)):
                    scoresWithUrl.append((urlCorpus[i][0], scores[i])) # dict url->score
                ranking = sortBy(scoresWithUrl, index=1, desc=True)
                ranking = [e[0] for e in ranking] # only urls, without scores
                currentRankings.append(ranking)
            yield (userId, currentRankings)

In [63]:
# We use MLIterator for multi-processing (33 hours of processing if not...):
mli = MLIterator(usersList, genFunct, logger=logger, printRatio=0.01, parallelProcesses=cpuCount())

With parallelProcesses > 1, this iterator is not consistent, meaning 2 iterations over same containers will not give items in the same order
30 containers to process.


In [64]:
warnFreeRAM(logger=logger)

22.19g of RAM remaining.


In [65]:
rankings = dict()
i = 0
modulo = int(len(evalData['candidates']) / 100) + 1
for (userId, currentRankings) in mli:
    rankings[userId] = currentRankings
    if i % modulo == 0:
        warnFreeRAM(logger=logger)
        if freeRAM() < 2:
            exit()
    i += 1

21.4g of RAM remaining.
  3% [                    ]
21.48g of RAM remaining.
21.5g of RAM remaining.
  6% [=                   ] (1m 58.02s left)
 10% [==                  ] (1m 16.23s left)
21.62g of RAM remaining.
 13% [==                  ] (55.51s left)
21.53g of RAM remaining.
 16% [===                 ] (43.25s left)
21.53g of RAM remaining.
 20% [====                ] (34.88s left)
21.46g of RAM remaining.
 23% [====                ] (29.538s left)
21.38g of RAM remaining.
21.38g of RAM remaining.
 26% [=====               ] (26.235s left)
 30% [======              ] (22.376s left)
21.53g of RAM remaining.
 33% [======              ] (19.38s left)
21.41g of RAM remaining.
 36% [=======             ] (17.186s left)
21.46g of RAM remaining.
 40% [========            ] (15.014s left)
21.43g of RAM remaining.
 43% [========            ] (13.443s left)
21.38g of RAM remaining.
 46% [=========           ] (12.251s left)
21.46g of RAM remaining.
 50% [==========          ] (10.78s left

In [66]:
config

{'splitVersion': 2,
 'k1': 1.8,
 'b': 0.75,
 'epsilon': 0.25,
 'maxUsers': 30,
 'maxDocuments': 10000,
 'useExtraNews': False,
 'minDF': 0.0005,
 'maxDF': 300,
 'lowercase': False,
 'doLemmatization': True,
 'implementation': 'BM25'}

In [53]:
candidates.keys()

dict_keys(['790664132', '2617447752', '62594694', '839264223741956096', '2786961721', '1198644368', '21379050', '97081361', '3223646504', '3075850549', '2724579924', '874445094384013312', '711144080084836352', '76975799', '2303865967', '981994399', '3027216186', '25023286', '37433390', '84137940', '86209862', '19086562', '3180163293', '1596810348', '22261364', '15984413', '621359253', '1447457215', '40872545', '729623'])

In [54]:
rankings.keys()

dict_keys(['15984413', '621359253', '1596810348', '1447457215', '22261364', '729623', '19086562', '37433390', '86209862', '84137940', '3027216186', '3180163293', '25023286', '40872545', '2303865967', '2724579924', '874445094384013312', '3223646504', '711144080084836352', '981994399', '97081361', '76975799', '3075850549', '839264223741956096', '21379050', '2617447752', '790664132', '2786961721', '1198644368', '62594694'])

In [67]:
checkRankings(rankings,candidates,maxUsers=None)

In [46]:
len(rankings[userId])

[['http://ow.ly/gnmh30hxxKj',
  'http://politi.co/2ma7rpw',
  'http://www.baltimoresun.com/news/maryland/environment/bs-md-air-lawsuit-20171228-story.html',
  'http://goo.gl/alerts/GT7wU',
  'http://nymag.com/daily/intelligencer/2018/01/conservatives-outsourced-climate-to-lobbyists-and-kooks.html',
  'https://www.gjsentinel.com/news/western_colorado/report-cites-climate-costs-of-gas-project/article_313e11dc-f7a3-11e7-b99b-10604b9f7e7c.html',
  'http://www.heraldnews.com/opinion/20180102/guest-opinion-good-side-of-environment-news-after-year-of-trump',
  'https://www.bloomberg.com/news/articles/2018-01-10/ford-sued-by-truck-owners-claiming-diesel-engines-were-rigged',
  'https://insideclimatenews.org/news/23052017/methane-rules-trump-epa-obama-clean-power-plan-waste-management-delay',
  'https://www.washingtonpost.com/politics/how-the-trump-era-is-changing-the-federal-bureaucracy/2017/12/30/8d5149c6-daa7-11e7-b859-fb0995360725_story.html?utm_term=.44c6991f607a',
  'https://www.postandco

In [47]:
len(candidates[userId])

1

In [68]:
addRanking(modelName, rankings, config, logger=logger)

# TF, IDF

In [11]:
def inition(docs):
    D = len(docs)
    avgdl = sum([len(doc)+ 0.0 for doc in docs]) / D # average length of document
    for doc in docs:
        tmp = {}
        for word in doc:
            tmp[word] = tmp.get(word, 0) + 1  # store the frequency of words temporarily
        f.append(tmp)
        for k in tmp.keys():
            tf[k] = tf.get(k, 0) + 1         # term frequency
    for k, v in tf.items():
        idf[k] = math.log(D - v + 0.5) - math.log(v + 0.5)
    return D, avgdl

# Similarity

In [12]:
def sim(doc, index):
    score = 0.0
    for word in doc:
        if word not in f[index]:
            continue
        d = len(document[index])
        score += (idf[word] * f[index][word] * (k1 + 1) / (f[index][word] + k1 * (1 - b + b * d / avgdl)))
    return score

In [13]:
def simall(doc):
    scores = []
    for index in range(D):
            score = sim(doc, index)
            scores.append(score)
    return scores

# Small test for gensim bm25

In [38]:
corpus = [
     ["black", "cat", "white", "cat"],
     ["cat", "outer", "space"],
     ["wag", "dog"]
]
bm25Model = bm25.BM25(corpus)

In [39]:
average_idf = sum(map(lambda k: float(bm25Model.idf[k]), bm25Model.idf.keys())) / len(bm25Model.idf.keys())

In [40]:
average_idf

0.4508818005689612

In [48]:
query_str = 'outer dog'
query = []
for word in query_str.strip().split():
    query.append(word)
scores = bm25Model.get_scores(query)
# scores.sort(reverse=True)
print(scores)


[0, 0.5108256237659907, 0.600971322077636]


In [108]:
def training():
    # 1.2<k1<2.0 step = 0.2
    # 0.5<b<0.8  step = 0.1      
    for k1 in [1.2,1.4,1.6,1.8,2.0]:       
        for b in [0.55,0.65,0.75]:
            config['k1'] = k1
            config['b'] = b
            rankings = dict()
            for (userId, currentRankings) in mli:
                rankings[userId] = currentRankings
            addRanking(modelName, rankings, config, logger=logger)